In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
 
 
lr = 0.01 #学习率
momentum = 0.5
log_interval = 10 #跑多少次batch进行一次日志记录
epochs = 10
batch_size = 64
test_batch_size = 1000
 
 
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Sequential(  # input_size=(1*28*28)
            nn.Conv2d(1, 6, 5, 1, 2),  # padding=2保证输入输出尺寸相同
            nn.ReLU(),  # input_size=(6*28*28)
            nn.MaxPool2d(kernel_size=2, stride=2),  # output_size=(6*14*14)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),  # input_size=(16*10*10)
            nn.MaxPool2d(2, 2)  # output_size=(16*5*5)
        )
        self.fc1 = nn.Sequential(
            nn.Linear(16 * 5 * 5, 120),
            nn.ReLU()
        )
        self.fc2 = nn.Sequential(
            nn.Linear(120, 84),
            nn.ReLU()
        )
        self.fc3 = nn.Linear(84, 10)
 
    # 定义前向传播过程，输入为x
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        # nn.Linear()的输入输出都是维度为一的值，所以要把多维度的tensor展平成一维
        x = x.view(x.size()[0], -1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x #F.softmax(x, dim=1)
 
 
 
def train(epoch):  # 定义每个epoch的训练细节
    model.train()  # 设置为trainning模式
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device)
        target = target.to(device)
        data, target = Variable(data), Variable(target)  # 把数据转换成Variable
        optimizer.zero_grad()  # 优化器梯度初始化为零
        output = model(data)  # 把数据输入网络并得到输出，即进行前向传播
        loss = F.cross_entropy(output,target)  #交叉熵损失函数
        loss.backward()  # 反向传播梯度
        optimizer.step()  # 结束一次前传+反传之后，更新参数
        if batch_idx % log_interval == 0:  # 准备打印相关信息
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))
 
 
def test():
    model.eval()  # 设置为test模式
    test_loss = 0  # 初始化测试损失值为0
    correct = 0  # 初始化预测正确的数据个数为0
    for data, target in test_loader:
 
        data = data.to(device)
        target = target.to(device)
        data, target = Variable(data), Variable(target)  #计算前要把变量变成Variable形式，因为这样子才有梯度
 
        output = model(data)
        test_loss += F.cross_entropy(output, target, size_average=False).item()  # sum up batch loss 把所有loss值进行累加
        pred = output.data.max(1, keepdim=True)[1]  # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()  # 对预测正确的数据个数进行累加
 
    test_loss /= len(test_loader.dataset)  # 因为把所有loss值进行过累加，所以最后要除以总得数据长度才得平均loss
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
 
 
 
if __name__ == '__main__':
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') #启用GPU
 
    train_loader = torch.utils.data.DataLoader(  # 加载训练数据
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))  #数据集给出的均值和标准差系数，每个数据集都不同的，都数据集提供方给出的
                       ])),
        batch_size=batch_size, shuffle=True)
 
    test_loader = torch.utils.data.DataLoader(  # 加载训练数据，详细用法参考我的Pytorch打怪路（一）系列-（1）
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,)) #数据集给出的均值和标准差系数，每个数据集都不同的，都数据集提供方给出的
        ])),
        batch_size=test_batch_size, shuffle=True)
 
    model = LeNet()  # 实例化一个网络对象
    model = model.to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)  # 初始化优化器
 
    for epoch in range(1, epochs + 1):  # 以epoch为单位进行循环
        train(epoch)
        test()
 
    torch.save(model, 'model.pth') #保存模型

100%|█████████████████████████████████████████████████████████████████████| 9912422/9912422 [04:50<00:00, 34148.63it/s]


Extracting ../data\MNIST\raw\train-images-idx3-ubyte.gz to ../data\MNIST\raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable



100%|████████████████████████████████████████████████████████████████████████| 28881/28881 [00:00<00:00, 116623.53it/s]


Extracting ../data\MNIST\raw\train-labels-idx1-ubyte.gz to ../data\MNIST\raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable



100%|███████████████████████████████████████████████████████████████████| 1648877/1648877 [00:01<00:00, 1181754.99it/s]


Extracting ../data\MNIST\raw\t10k-images-idx3-ubyte.gz to ../data\MNIST\raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable



100%|██████████████████████████████████████████████████████████████████████████| 4542/4542 [00:00<00:00, 875322.95it/s]


Extracting ../data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ../data\MNIST\raw

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.283764
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.316323
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.289027
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.293226
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.283987
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.288660
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.287969
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.273998
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.280258
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.269363
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.261899
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.253360
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.245652
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.243019
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.236249
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.183898
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.177444
Train Epoch: 1 [10880/60000 (18%)]	Loss: 2.089804
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.015273
Tr

D:\ANACONDA\envs\pytorch\Lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.1451, Accuracy: 9548/10000 (95%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.207624
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.172717
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.162557
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.205533
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.084554
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.109190
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.113450
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.083568
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.122562
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.095646
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.179044
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.071510
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.178948
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.083553
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.130438
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.131631
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.083445
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.080863
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.132294
Train Epoch: 2 [12